In [ ]:
import requests
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from datetime import datetime
from src.Client import Client
from src.Server import Server

In [ ]:
server_url = "http://127.0.0.1:5000"  # Server endpoint


In [ ]:
node_file = "./Datasets/node8.csv"
data = pd.read_csv(node_file)

# Extract features (all columns except the 9th one)
local_dataset = data.drop(columns=data.columns[9]).values  # Drop the 9th column for features

# Extract labels (9th column)
targetvar = data.iloc[:, 9].values  # Select the 9th column for labels

client8=  Client(
        client_id="client8", 
        local_data=local_dataset,
        local_labels=targetvar,
        address="Nagpur ,India"
    ) 



# Print shapes to verify
print(f"Local Data Shape: {client8.local_data.shape}")
print(f"Local Labels Shape: {client8.local_labels.shape}")

Local Data Shape: (4380, 16)
Local Labels Shape: (4380,)


In [ ]:
# Fetch global model from the server
response = requests.get(f"{server_url}/get_global_model")
global_model = response.json()

In [ ]:
# Train local model
local_model = LinearRegression()
local_model.coef_ = np.array(global_model["coef_"])
local_model.intercept_ = global_model["intercept_"]
local_model.fit(local_data, local_labels)


In [ ]:
# Evaluate the local model
y_pred = local_model.predict(local_data)
mse = mean_squared_error(local_labels, y_pred)
print(f"{client_id} - Local MSE: {mse:.4f}")

In [ ]:
# Send update to the server
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
update = {
    "client": {"client_id": client_id, "latitude": latitude, "longitude": longitude},
    "model": {"coef_": local_model.coef_.tolist(), "intercept_": local_model.intercept_},
    "metadata": {"weight": 1, "timestamp": timestamp}
}
response = requests.post(f"{server_url}/send_update", json=update)
print(response.json())

In [ ]:
# Request aggregation (optional)
requests.post(f"{server_url}/aggregate")
